<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# A gentle introduction

`apply_ufunc` is a more advanced wrapper that is designed to apply functions
that expect and return NumPy (or other arrays). For example, this would include
all of SciPy's API. Since `apply_ufunc` operates on lower-level NumPy or Dask
objects, it skips the overhead of using Xarray objects making it a good choice
for performance-critical functions. Xarray uses `apply_ufunc` internally
to implement much of its API, meaning that it is quite powerful!

Learning goals:
- Learn that `apply_ufunc` automates aspects of applying computation functions that are designed for pure arrays (like numpy arrays) on xarray objects

## Setup

In [ ]:
import numpy as np
import xarray as xr

xr.set_options(display_expand_data=False)

Let's load a dataset

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

## A simple example

Simple functions that act independently on each value should work without any
additional arguments. 

Consider the following `squared_error` function

In [ ]:
def squared_error(x, y):
    return (x - y) ** 2

We can apply this manually by extracting the underlying numpy array

In [ ]:
numpy_result = squared_error(ds.air.data, 1)
numpy_result

To convert this result to a DataArray, we could do it manually

In [ ]:
xr.DataArray(data=numpy_result, dims=ds.air.dims, coords=ds.air.coords)

A shorter version uses [DataArray.copy](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.copy.html)

In [ ]:
ds.air.copy(data=numpy_result)

Using `DataArray.copy` works for such simple cases but doesn't generalize that well. For example, consider a function that removed one dimension and added a new dimension.

`apply_ufunc` can handle such cases. Here's how to use it with `squared_error`

In [ ]:
xr.apply_ufunc(squared_error, ds.air, 1)

## How does apply_ufunc work?

To illustrate how `apply_ufunc` works, let us write a small wrapper function. This will let us examine what data is received and returned from the applied function. 

```{tip}
This trick is very useful for debugging
```


In [ ]:
def wrapper(x, y):
    print(f"received x of type {type(x)}, shape {x.shape}")
    print(f"received y of type {type(y)}")
    return squared_error(x, y)


xr.apply_ufunc(wrapper, ds.air, 1)

We see that `wrapper` receives the underlying numpy array (`ds.air.data`), and the integer `1`. 

Essentially, `apply_ufunc` does the following:
1. extracts the underlying array data, 
2. passes it to the user function, 
3. receives the returned values, and 
4. then wraps that back up as an array

apply_ufunc easily handles both dataarrays and datasets. 

When passed a Dataset, apply-ufunc will loop over the data variables and sequentially pass those to `squared_error`. So `squared_error` always receives a numpy array

In [ ]:
xr.apply_ufunc(wrapper, ds, 1)

In [ ]:
xr.apply_ufunc(squared_error, ds, 1)

## Reductions and core dimensions

`squared_error` operated on a per-element basis. How about a reduction like `np.mean`?

Such functions involve the concept of "core dimensions". One way to think about core dimensions is to consider the smallest dimensionality of data necessary to apply the function.

For using more complex operations that consider some array values collectively,
it’s important to understand the idea of **core dimensions**. 
Usually, they correspond to the fundamental dimensions over
which an operation is defined, e.g., the summed axis in `np.sum`. A good clue
that core dimensions are needed is the presence of an `axis` argument on the
corresponding NumPy function.

Let's write a function that computes the mean along `time` for a provided xarray object. This function requires one core dimension `time`. For `ds.air` note that `time` is the 0th axis.

In [ ]:
ds.air.dims

In [ ]:
np.mean(ds.air, axis=ds.air.get_axis_num("time"))

In [ ]:
np.mean(ds.air.data, axis=0)

Let's try to use `apply_ufunc` to replicate `np.mean(ds.air.data, axis=0)`

In [ ]:
xr.apply_ufunc(
    # function to apply
    np.mean,
    # object with data to pass to function
    ds,
    # keyword arguments to pass to np.mean
    kwargs={"axis": 0},
)

The error here
> applied function returned data with unexpected number of dimensions. Received 2 dimension(s) but expected 3 dimensions with names: ('time', 'lat', 'lon')

means that while `np.mean` did indeed reduce one dimension, we did not tell `apply_ufunc` that this would happen. That is, we need to specify the core dimensions on the input.

In [ ]:
xr.apply_ufunc(
    np.mean,
    ds,
    # specify core dimensions as a list of lists
    # here 'time' is the core dimension on `ds`
    input_core_dims=[["time"]],
    kwargs={"axis": 0},
)

This next error is a little confusing.

> size of dimension 'lat' on inputs was unexpectedly changed by applied function from 25 to 53. Only dimensions specified in ``exclude_dims`` with xarray.apply_ufunc are allowed to change size.


A good trick here is to pass a little wrapper function to `apply_ufunc` instead and inspect the shapes of data received by the wrapper.


In [ ]:
def wrapper(array, **kwargs):
    print(f"received {type(array)} shape: {array.shape}, kwargs: {kwargs}")
    result = np.mean(array, **kwargs)
    print(f"result.shape: {result.shape}")
    return result


xr.apply_ufunc(
    wrapper,
    ds,
    # specify core dimensions as a list of lists
    # here 'time' is the core dimension on `ds`
    input_core_dims=[["time"]],
    kwargs={"axis": 0},
)

Now we see the issue:

    received <class 'numpy.ndarray'> shape: (25, 53, 2920), kwargs: {'axis': 0}
    result.shape: (53, 2920)
    
The `time` dimension is of size `2920` and is now the last axis of the array but was initially the first axis

In [ ]:
ds.air.get_axis_num("time")

This illustrates an important concept: **arrays are transposed so that core dimensions are at the end**. 

With `apply_ufunc`, core dimensions are recognized by name, and then moved to
the last dimension of any input arguments before applying the given function.
This means that for functions that accept an `axis` argument, you usually need
to set `axis=-1`

Such behaviour means that our functions (like `wrapper` or `np.mean`) do not need to know the exact order of dimensions. They can rely on the core dimensions being at the end allowing us to write very general code! 

We can fix our `apply_ufunc` call by specifying `axis=-1` instead.

In [ ]:
def wrapper(array, **kwargs):
    print(f"received {type(array)} shape: {array.shape}, kwargs: {kwargs}")
    result = np.mean(array, **kwargs)
    print(f"result.shape: {result.shape}")
    return result


xr.apply_ufunc(
    wrapper,
    ds,
    input_core_dims=[["time"]],
    kwargs={"axis": -1},
)

## Exercise

Use `apply_ufunc` to apply `sp.integrate.trapz` along the `time` axis.


In [ ]:
import scipy as sp
import scipy.integrate

xr.apply_ufunc(scipy.integrate.trapz, ds, input_core_dims=[["time"]], kwargs={"axis": -1})